In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7,google_api_key=os.getenv("GEMINI_API_KEY"))